In [1]:
%pylab

import serial
import serial.tools.list_ports

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
for port in serial.tools.list_ports.comports():
    print(port.device)

/dev/cu.Bluetooth-Incoming-Port
/dev/cu.usbmodem1411


In [17]:
### Colton's Testing Code
###Code to find the grid of temperatures at one point in time. Will use to try to calibrate the sensors.

import serial, sys
from pylab import *
import csv

try:
    #port = '/dev/ttyUSB1'
    port = '/dev/cu.usbmodem1411'
    #port = sys.argv[3]
    ser = serial.Serial(port, 57600, timeout=2)
except:
    print( "Could not open port %s, exiting.") % port
    exit(0)
        
# set up matrix for temperature data
grid = zeros([10,10])
newgrid = zeros([10,10])
subgrid = zeros([10,10])
subtracts = []

#Open the file for the subtraction grid that will be used to calibrate, and convert the grid into a list.
with open('2D-SubtractionGrid-2.csv', mode='r') as subtract_file:
    subtract_reader = csv.reader(subtract_file, delimiter=' ')

    for row in subtract_reader:
        for item in row:
            subtracts.append(float(item))
        
# read and discard a line to help get things in sync.
junk = ser.readline()

# Read a line of temperature data from the Arduino
line = ser.readline()
ser.flushInput()
# split the line into a list of floating-point values
temperatures = [float(T) for T in line.split()]

# sort the list of temperatures into a matrix for plotting. Create the newgrid (calibrated).
for j in range(10):
    for k in range(10):
        grid[j,k] = temperatures[j*10+k]
        subgrid[j,k] = subtracts[j*10+k]
        
        newgrid[j,k] = grid[j,k] + subgrid[j,k]
        
        
newgrid



array([[ 19.88,  19.81,  19.75,  19.62,  19.44,  19.44,  19.68,  19.81,
         19.81,  20.37],
       [ 19.87,  19.69,  19.5 ,  19.37,  19.25,  19.12,  19.31,  19.44,
         19.81,  19.94],
       [ 19.63,  19.56,  19.19,  19.06,  18.56,  18.69,  18.88,  19.31,
         19.5 ,  19.69],
       [ 19.56,  19.25,  18.75,  18.5 ,  18.06,  18.  ,  18.44,  19.25,
         19.25,  19.69],
       [ 19.44,  19.13,  18.75,  18.  ,  17.5 ,  17.5 ,  18.  ,  18.75,
         19.25,  19.56],
       [ 19.44,  19.06,  18.63,  18.06,  17.5 ,  17.5 ,  18.06,  18.81,
         19.13,  19.44],
       [ 19.32,  19.25,  18.88,  18.44,  18.06,  18.  ,  18.37,  18.94,
         19.25,  19.56],
       [ 19.57,  19.5 ,  19.07,  18.57,  18.56,  18.5 ,  18.75,  19.19,
         19.56,  19.69],
       [ 20.19,  19.63,  19.38,  19.25,  19.06,  19.13,  19.31,  19.32,
         19.62,  19.69],
       [ 19.68,  19.69,  19.56,  19.38,  19.32,  19.31,  19.38,  19.5 ,
         19.81,  19.82]])

In [5]:
"""This function calibrates the data from the plate using the given subtraction grid.
        
    Args:
        grid (array): temperature data from the 2-d heat flow apparatus at the current time in grid format
        subtractiongrid (array): grid used to calibrate the current temperature grid
        
    Returns:
        Calibrated temperature data in the form of a grid.
    """


def newgridfunc(grid, subtractiongrid):
    
    newgrid = zeros([10,10])
    
    for j in range(10):
            for k in range(10):
                newgrid[j,k] = grid[j,k] + subtractiongrid[j,k]

    return newgrid

In [6]:
xvec = [1.7, 4.7, 7.7, 10.7, 13.7, 16.7, 19.7, 22.7, 25.7, 28.7]
yvec = [1.7, 4.7, 7.7, 10.7, 13.7, 16.7, 19.7, 22.7, 25.7, 28.7]

xgrid, ygrid = meshgrid(xvec, yvec)

In [15]:
#!/usr/bin/env python

'''
T_reader.py
Eric Ayars
5/22/12

Program to capture and plot data from the 10x10 temperature sensor array.

'''

import serial, sys
from pylab import *
import csv
import time
import os

def TempPlot(minT, maxT):

    #import serial, sys ---> Colton Commented this out
    #from pylab import * ---> Colton Commented this out

    size = 12           # size of display window
    
    try:
        minT = float(sys.argv[1])
        maxT = float(sys.argv[2])
    except (ValueError, IndexError):
        print( '''
    Call program with three parameters: 
        minimum expected temperature, 
        maximum expected temperature,
        location of serial port.
    ''')
        exit(0)

    # start the serial port
    try:
        #port = '/dev/ttyUSB1'
        port = '/dev/cu.usbmodem1411'
        #port = sys.argv[3]
        ser = serial.Serial(port, 57600, timeout=2)
    except:
        print( "Could not open port %s, exiting.") % port
        exit(0)

    # allow animation
    ion()

    # set up matrix for temperature data
    grid = zeros([10,10])
    newgrid = zeros([10,10])
    subgrid = zeros([10,10])
    
    subtracts = []
    timelist = []
    #templist = []

    #starttime = time.time() # Starting reference time
    
    with open('Trial1-subtractgrid.csv', mode='r') as subtract_file:
        subtract_reader = csv.reader(subtract_file, delimiter=' ')

        for row in subtract_reader:
            for item in row:
                subtracts.append(float(item))
                
    for j in range(10):
        for k in range(10):
            subgrid[j,k] = subtracts[j*10+k]
    

    filename = input('Name of File? ')  #filename.csv
    exists = os.path.isfile(filename)
    while exists == True:
        print('This File name already exists!')
        filename = input('Name of File? ')  #filename.csv
        exists = os.path.isfile(filename)
        
    file_object  = open(filename, 'w')    
    
    
    
    # read and discard a line to help get things in sync.
    junk = ser.readline()

    # open the view window
    fig = figure(figsize=(size, size))

    # define the levels at which to draw contours. The resolution 
    # of the sensors at 9-bit (default) sensitivity is about 0.12,
    # so setting level spacing below this does not help.
    levels = arange(minT, maxT, 0.12)

    
    #Beginning of the time
    starttime = time.time() # Starting reference time
    
    # now the main loop, which continues until an error occurs.
    while True:

        try:
            
            ValError = False
            # this is the stuff to do unless a problem occurs.

            # Read a line of temperature data from the Arduino
            line = ser.readline()
            ser.flushInput()
            # split the line into a list of floating-point values
            temperatures = [float(T) for T in line.split()]

            # sort the list of temperatures into a matrix for plotting
            for j in range(10):
                for k in range(10):
                    grid[j,k] = temperatures[j*10+k]


            newgrid = newgridfunc(grid, subgrid)

            
            
                    
            # plot the data
            fig.clear()
            #tempplot = contourf(newgrid, levels, cmap = cm.coolwarm)
            tempplot = contourf(xgrid, ygrid, newgrid, levels, cmap = cm.coolwarm) #Trying to plot whole plate
            tempbar = fig.colorbar(tempplot)
            tempbar
            tempplot
            
            plt.xlabel('X index of sensor')
            plt.ylabel('Y index of sensor')
            plt.title('Temperature along Plate')
            plt.ylim(0,30)
            plt.xlim(0,30)
            
            tempbar.set_label('Temperature Range (Celsius)')
            
            draw()
            plt.pause(.01)
            
            
            endtime = time.time() #time after a plot
            elapsedtime = endtime - starttime
            
            #Saving the time and temp data to the file
            if elapsedtime >= 1:
                #datalist = []
                #datalist.append(elapsedtime)
                #datalist.append(newgrid)
                #timelist.append(elapsedtime)
                #templist.append(newgrid)
                print(elapsedtime)
                
                    
                deltatime = str(elapsedtime)
                file_object.write(deltatime + '\n')
                for row in newgrid:
                    for num in row:
                        temp = str(num)
                        file_object.write(temp + ' ')
                file_object.write('\n')
            
        # Handle problems here
        except ValueError:
            # error at float conversion
            print( "Conversion error: frame dropped, continuing collection.")
            ValError = True
            continue
        except IndexError:
            # not all values received for a measurement set
            print( "Partial frame received: frame dropped, continuing collection.")
            continue
        except serial.serialutil.SerialException:
            # Someone just unplugged the device, or other loss of communication.
            print( "Device disconnected, exiting.")
            break
        except: 
            # Catch-all
            print( "Unknown error! Exiting program...")
            junk = raw_input("Press Enter to continue.")
            break

    return print("done")

In [16]:
#Use this to call the tempplot function. First argument is the min temp and the second is the max temp.

# Put .csv at the end of file name

TempPlot(-5, 25)



    Call program with three parameters: 
        minimum expected temperature, 
        maximum expected temperature,
        location of serial port.
    
Name of File? Trial1-Exp-plateplots
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
Partial frame received: frame dropped, continuing collection.
P

NameError: name 'raw_input' is not defined